Tutorial :
https://docs.ragas.io/en/stable/getstarted/evaluation.html

Import

In [3]:
import sys

sys.path.append('../')
sys.path.append("../..")

import os
import openai
import pandas as pd
from Call_RAG import ask
from datasets import Dataset
from dotenv import load_dotenv, find_dotenv

%matplotlib inline


env_path = find_dotenv()

_ = load_dotenv(find_dotenv())  # read local .env file

openai.api_key = os.environ["OPENAI_API_KEY"]

Upload evaluation dataset

In [4]:
#df_testset = pd.read_csv("evaluation_dataset.csv")
df_testset = pd.read_csv("evaluation_dataset_LLAMA3.csv")

Add answer column from RAG

In [6]:
# Ajouter une colonne 'answer' en initialisant avec des chaînes vides ou NaN
df_testset["answer"] = pd.NA

# Générer la réponse pour chaque question et la stocker dans la colonne 'answer'
for index, row in df_testset.iterrows():
    user_input = row["question"]  # Assurez-vous que la colonne s'appelle 'question'
    output = ask(user_input)
    df_testset.at[index, "answer"] = output

df_testset.to_csv("evaluation_dataset for RAG1.csv", index=False)

Transform df_testset into dictionnary then into dataset

In [7]:
from ragas import evaluate

Dictionnary_from_dataset = {
    "question": df_testset["question"].tolist(),
    "answer": df_testset["answer"].tolist(),
    "contexts": df_testset["contexts"]
    .apply(lambda x: x.split(","))
    .tolist(),  # Chaque contexte est une liste de chaînes
    "ground_truth": df_testset["ground_truth"].tolist(),
}

dataset = Dataset.from_dict(Dictionnary_from_dataset)

ArrowTypeError: Expected bytes, got a 'float' object

Launch evaluation

In [6]:
result = evaluate(dataset)

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

Display result of the evaluation

In [7]:
df = result.to_pandas()
df.head()

,question,answer,contexts,ground_truth,answer_relevancy,context_precision,faithfulness,context_recall
0,How do the Aichi Targets promote coordinated a...,The Aichi Targets promote coordinated actions ...,[[' are promoted through the Aichi Targets. Th...,The Aichi Targets promote coordinated actions ...,0.928637,1.000000,0.4,1.0
1,What is the purpose of the Observatory of Biod...,The purpose of the Observatory of Biodiversity...,[['MINFOF: Ministère des Forêts et de la Faune...,The purpose of the Observatory of Biodiversity...,1.000000,1.000000,1.0,1.0
2,How do protected areas in Central African coun...,Protected areas in Central African countries c...,"[['40\nVarious COMIFAC directives, notably th...",Protected areas in Central African countries c...,1.000000,0.849645,1.0,1.0
3,How does habitat destruction pose a threat to ...,Habitat destruction in Central Africa leads to...,[['32\n2.2. Rich but threatened \nanimal dive...,Habitat destruction poses a threat to great ap...,0.921716,0.863277,1.0,1.0
4,What were the broad objectives for the overhau...,The broad objectives for the overhaul of the C...,[['42\nand to clarify its positioning in the n...,The broad objectives for the overhaul of UTOs ...,1.000000,0.923989,0.5,1.0
